In [1]:
#Today you are creating a recommender where the user inputs a song title and check if that song is in the list you created.  
#If it is,  give a different random song and artist from the list.  
#If it is not on the list, let the user know that you have no recommendation at this time.

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime

In [3]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [4]:
response = requests.get(url)

In [5]:
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content, "html.parser")
# soup

In [7]:
song_elements=soup.find_all('div', class_='chart-content')

In [8]:
if not song_elements:
    print("No song elements found. Please check if the website structure has changed.")
    exit()

data = []
for i, element in enumerate(song_elements):
    title = element.find('cite', class_='title').text.strip()
    artist = element.find('em', class_='artist').text.strip()

    # Extracting genre
    genre = "Genre not found"
    ul_tag = element.find('ul')
    if ul_tag:
        for li_tag in ul_tag.find_all('li'):
            if 'Genre' in li_tag.text:
                genre = li_tag.text.strip().replace('Genre:', '')
                break

   # Extracting release date using regular expression
    release_date_text = ""
    release_date_match = re.search(r'Release Date: (\w+ \d{1,2}, \d{4})', element.get_text())
    if release_date_match:
        release_date_text = release_date_match.group(1)
        
        # Convert release date to date format
        release_date = datetime.strptime(release_date_text, "%B %d, %Y").strftime("%d/%m/%Y")

    data.append([title, artist, genre, release_date])

df = pd.DataFrame(data, columns=['title', 'artist', 'genre', 'release_date'])
df.head()

,title,artist,genre,release_date
0,TEXAS HOLD 'EM,Beyoncé,Country,11/02/2024
1,Lose Control,Teddy Swims,Pop,23/06/2023
2,Beautiful Things,Benson Boone,Pop,19/01/2024
3,Flowers,Miley Cyrus,Pop,12/01/2023
4,Turn the Lights Back On,Billy Joel,Pop,01/02/2024


In [9]:
# first ask for a songfrom user
# if title in df then dive new song from df
# if title not in df answer politely 

In [17]:
def recommend_song(df, input_song):
    # Convert input to lowercase for case-insensitive comparison
    input_song = input_song.lower()
    
    # Check if the input song is in the DataFrame
    if input_song in df['title'].str.lower().values:
        # Get a random song that is not the input song
        random_song = df[df['title'].str.lower() != input_song].sample(n=1)
        recommended_song = random_song.iloc[0]['title']
        recommended_artist = random_song.iloc[0]['artist']
        print(f"Thanks for answering {input_song}. Based on your input, we reccomend {recommended_song} by {recommended_artist}.")
    else:
        print("We have no recommendation at this time.")

In [21]:
user_input = input("Enter a song title: ")
recommend_song(df, user_input)

Enter a song title: beautiful things
Thanks for answering beautiful things. Based on your input, we reccomend Love Story (Taylor’s Version) by Taylor Swift.
